### Import Libraries

In [4]:
import os
from textwrap import dedent
from dotenv import load_dotenv
import pandas as pd
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langchain_openai import AzureChatOpenAI
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import markdown

# Load environment variables
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), verbose=True, override=True)

# Load environment variables
load_dotenv()

os.environ["AZURE_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_ENDPOINT"] = os.getenv("AZURE_ENDPOINT")
os.environ["AZURE_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION") 
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")


### Generate Markdown file for Balance Sheet

In [5]:
## Generate Markdown file
def extract_text_from_excel(file, sheetnames):
    if len(sheetnames)>0:
        for sheet in sheetnames:
            df = pd.read_excel(file, engine='openpyxl', sheet_name=sheet)
            df_cleaned = df.dropna(how='all')
            df2 = df_cleaned.fillna('').reset_index(drop=True)
            with open("../data/processed_inputs/profit_loss.md", "a") as f:
                f.write(f"""##### {sheet} \n{str(df2.to_markdown())}\n""")
                # f.write(str(df2.to_markdown()))
    else:
        print("SheetNames not provided")

In [6]:
file_path = '../data/1. CMA_Data.xlsx'
extract_text_from_excel(file_path,sheetnames=['Profit & Loss Statement'])

### Define the LangGraph Agent for BalanceSheet Analyzer

In [7]:
class AgentState(TypedDict):
    question: str
    data: str
    result: str 


def profit_loss_analyzer_agent(state: AgentState):
    """
    Financial Analyzer Agent takes input questions and responds to the user's questions by processing the image provided

    Returns:
    dict: The response from the LLM.
    """
    # Input User Question
    question = state["question"]

    with open("../data/processed_inputs/profit_loss.md", "r") as f:
        state['data'] = f.read()

    data = state['data']
    
    system = dedent(""" 
    You are a Financial Analyst. Analyze the Give Profit and Loss Data and help answer questions based on the Data.
                    
    Identify the most important financial metrics and ratios that provide insights into a company’s profitability, revenue generation, 
    and cost management. Discuss how these metrics impact the company’s overall financial performance. Focus on the following:

    Key metrics that should be monitored in a Profit and Loss (P&L) Statement :
            1. Revenue/Sales: Assess total revenue, breaking it down by product lines or business segments. Consider growth trends and seasonal variations.
            2. Cost of Goods Sold (COGS): Analyze direct production costs. Discuss efficiency in managing production costs and its impact on profit margins. 
            3. Calculate and assess gross profit margin: Gross Profit Margin = (Gross Profit / Revenue), which helps to metric reveals how well the company is managing its production costs relative to its sales.
            4. Operating Expenses: Review operating expenses, including SG&A, R&D, and marketing. Assess the balance in spending.
            5. Operating Income (EBIT): Analyze operating income (Earnings Before Interest and Taxes) and its trend. Determine operational efficiency excluding interest and taxes.
            6. Operating Profit Margin: Calculate operating profit margin: Operating Profit Margin = (EBIT/Revenue)
            7. Net Income: Review net income (bottom line) reflecting total profitability after all expenses. Assess overall financial health.
            8. Net Profit Margin: Calculate net profit margin:  Net Profit Margin = (Net Income / Revenue)
            9. Earnings Per Share (EPS): If applicable, analyze EPS to evaluate profit per outstanding share. Compare EPS trends over time.
            10. EBITDA: Evaluate Earnings Before Interest, Taxes, Depreciation, and Amortization as a measure of operational performance and cash flow generation.
            11. Depreciation and Amortization: Examine the impact of depreciation and amortization on net income, especially for capital-intensive companies.
            12. Interest Expense: Review the cost of debt and its effect on profitability. High interest expenses may indicate over-leverage.
            13. Tax Expense: Assess tax liability and its impact on net income. Look for changes in tax rates or strategies.
            14. EBIT to EBITDA Conversion: Understand the relationship between EBIT and EBITDA and how non-cash expenses (depreciation, amortization) affect profitability.
    """)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", f"Profit-Loss Data: {data}"),
            ("human", f"Question: {question}"),
        ]
    )

    # Initialize the LLM
    llm = AzureChatOpenAI(
        model="gpt-4o-mini",
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_version=os.getenv("AZURE_API_VERSION"),
        temperature=0
    )

    fin_analyst = prompt | llm | StrOutputParser()

    result = fin_analyst.invoke({"question": question, "profit_loss_data": data})

    # Update the state with the result
    state["result"] = result

    return state

workflow = StateGraph(AgentState)

workflow.add_node("profit_loss_analyzer_node", profit_loss_analyzer_agent)

workflow.set_entry_point("profit_loss_analyzer_node")

app = workflow.compile()

### User Questions

In [8]:
user_question_1 = "Give the net profit mentioned in the PL statement in percentage?"
result = app.invoke({"question": user_question_1})
print(result["result"])

To calculate the net profit margin as a percentage, we use the formula:

\[
\text{Net Profit Margin} = \left( \frac{\text{Net Profit}}{\text{Net Sales}} \right) \times 100
\]

From the Profit & Loss statement, we can extract the following values for the year ending 31.03.2023:

- **Net Profit**: 85.94 lacs
- **Net Sales**: 1189.25 lacs

Now, substituting these values into the formula:

\[
\text{Net Profit Margin} = \left( \frac{85.94}{1189.25} \right) \times 100 \approx 7.24\%
\]

Thus, the net profit margin for the year ending 31.03.2023 is approximately **7.24%**.


In [29]:
user_question_2 = "What is the Operating Profit Margin?"
result_2 = app.invoke({"question": user_question_2})
print(result_2["result"])

To calculate the Operating Profit Margin, we use the formula:

\[ \text{Operating Profit Margin} = \frac{\text{Operating Income (EBIT)}}{\text{Revenue}} \]

From the provided Profit and Loss data, we can extract the following values:

1. **Operating Income (EBIT)** for the year ending 31.03.2023: **103.16 lacs**
2. **Net Sales (Revenue)** for the year ending 31.03.2023: **1189.25 lacs**

Now, substituting these values into the formula:

\[ \text{Operating Profit Margin} = \frac{103.16}{1189.25} \]

Calculating this gives:

\[ \text{Operating Profit Margin} \approx 0.0867 \text{ or } 8.67\% \]

Thus, the Operating Profit Margin for the year ending 31.03.2023 is approximately **8.67%**. This indicates that for every rupee of revenue generated, the company retains about 8.67 paise as operating profit before interest and taxes.


In [30]:
user_question_3 = "What is the Gross Profit Margin?"
result_3 = app.invoke({"question": user_question_3})
print(result_3["result"])

To calculate the Gross Profit Margin, we use the formula:

**Gross Profit Margin = (Gross Profit / Revenue) x 100**

From the provided Profit and Loss data, we can extract the following values:

- **Gross Profit** for the year ending 31.03.2023: **261.45 lacs**
- **Net Sales (Revenue)** for the year ending 31.03.2023: **1189.25 lacs**

Now, we can calculate the Gross Profit Margin:

\[
\text{Gross Profit Margin} = \left( \frac{261.45}{1189.25} \right) \times 100
\]

Calculating this gives:

\[
\text{Gross Profit Margin} = \left( 0.2201 \right) \times 100 \approx 22.01\%
\]

Thus, the Gross Profit Margin for the year ending 31.03.2023 is approximately **22.01%**. 

This metric indicates that the company retains about 22.01% of its revenue as gross profit after accounting for the cost of goods sold, which is a positive sign of profitability and efficiency in managing production costs relative to sales.


In [31]:
user_question_4 = "What is the Earnings Before Interest and Taxes (EBIT)"
result_4 = app.invoke({"question": user_question_4})
print(result_4["result"])

Earnings Before Interest and Taxes (EBIT) can be calculated from the Profit and Loss statement by taking the Operating Profit, which is already provided in the data.

From the Profit and Loss data, the Operating Profit (which is equivalent to EBIT) for the years is as follows:

- **31.03.2022**: 36.08 lacs
- **31.03.2023**: 103.16 lacs
- **31.03.2024**: 176.02 lacs
- **31.03.2025**: 231.11 lacs
- **31.03.2026**: 282.74 lacs
- **31.03.2027**: 332.14 lacs
- **31.03.2028**: 379.48 lacs
- **31.03.2029**: 425.13 lacs

Thus, the EBIT figures for the respective years are:

- **2022**: 36.08 lacs
- **2023**: 103.16 lacs
- **2024**: 176.02 lacs
- **2025**: 231.11 lacs
- **2026**: 282.74 lacs
- **2027**: 332.14 lacs
- **2028**: 379.48 lacs
- **2029**: 425.13 lacs

These figures indicate the company's operational performance before accounting for interest and taxes.


In [32]:
user_question_5 = "What is the Return on Sales (ROS) for each year?"
result_5 = app.invoke({"question": user_question_5})
print(result_5["result"])

Return on Sales (ROS) is a financial metric that measures a company's operational efficiency and profitability relative to its total sales. It is calculated using the formula:

\[ \text{Return on Sales (ROS)} = \frac{\text{Operating Profit}}{\text{Net Sales}} \times 100 \]

From the provided Profit and Loss data, we can extract the Operating Profit and Net Sales for each year:

- **Operating Profit** (from row 27)
- **Net Sales** (from row 6)

Now, let's calculate the ROS for each year:

1. **For 31.03.2022:**
   - Operating Profit: 36.08 lacs
   - Net Sales: 599.96 lacs
   - ROS = (36.08 / 599.96) * 100 = 6.01%

2. **For 31.03.2023:**
   - Operating Profit: 103.16 lacs
   - Net Sales: 1189.25 lacs
   - ROS = (103.16 / 1189.25) * 100 = 8.68%

3. **For 31.03.2024:**
   - Operating Profit: 176.02 lacs
   - Net Sales: 1274.45 lacs
   - ROS = (176.02 / 1274.45) * 100 = 13.80%

4. **For 31.03.2025:**
   - Operating Profit: 231.11 lacs
   - Net Sales: 1318.83 lacs
   - ROS = (231.11 / 1318.8

In [33]:
user_question_6 = "What is the Cost of Goods Sold (COGS) as a percentage of Net Sales?"
result_6 = app.invoke({"question": user_question_6})
print(result_6["result"])

To calculate the Cost of Goods Sold (COGS) as a percentage of Net Sales, we can use the following formula:

\[
\text{COGS Percentage} = \left( \frac{\text{COGS}}{\text{Net Sales}} \right) \times 100
\]

From the provided Profit and Loss data, we can extract the following values:

- **Net Sales** for the year ending 31.03.2023: 1189.25 lacs
- **COGS** for the year ending 31.03.2023: 927.80 lacs

Now, substituting these values into the formula:

\[
\text{COGS Percentage} = \left( \frac{927.80}{1189.25} \right) \times 100
\]

Calculating this gives:

\[
\text{COGS Percentage} = \left( 0.7805 \right) \times 100 \approx 78.05\%
\]

Therefore, the Cost of Goods Sold (COGS) as a percentage of Net Sales for the year ending 31.03.2023 is approximately **78.05%**.


In [34]:
user_question_7 = dedent("""Calculate and give the below by each year 
                            1. Revenue/Sale
                            2. Cost of Goods Sold(COGS)
                            3. Gross Profit Margin
                            4. Operating Expenses
                            5. Operating Income (EBIT)""")
result_7 = app.invoke({"question": user_question_7})
print(result_7["result"])

Based on the provided Profit and Loss data, we can calculate the required metrics for each year. Below are the calculations for Revenue/Sales, Cost of Goods Sold (COGS), Gross Profit Margin, Operating Expenses, and Operating Income (EBIT).

### 1. Revenue/Sales
- **Net Sales** (from the P&L statement):
  - 31.03.2022: 599.96 lacs
  - 31.03.2023: 1189.25 lacs
  - 31.03.2024: 1274.45 lacs
  - 31.03.2025: 1318.83 lacs
  - 31.03.2026: 1361.42 lacs
  - 31.03.2027: 1404.03 lacs
  - 31.03.2028: 1446.63 lacs
  - 31.03.2029: 1489.23 lacs

### 2. Cost of Goods Sold (COGS)
- **COGS** (from the P&L statement):
  - 31.03.2022: 466.95 lacs
  - 31.03.2023: 927.80 lacs
  - 31.03.2024: 958.94 lacs
  - 31.03.2025: 967.69 lacs
  - 31.03.2026: 978.09 lacs
  - 31.03.2027: 990.74 lacs
  - 31.03.2028: 1005.44 lacs
  - 31.03.2029: 1021.84 lacs

### 3. Gross Profit Margin
- **Gross Profit Margin** = (Gross Profit / Revenue) * 100
- **Gross Profit** = Revenue - COGS
- Calculations:
  - 31.03.2022: Gross Profit 

In [35]:
user_question_8 = "Tell me the percentage of Growth between years 2023 and 205"
result_8 = app.invoke({"question": user_question_8})
print(result_8["result"])

To calculate the percentage growth between the years 2023 and 2025 for the Net Sales and Net Profit, we can use the following formula:

\[
\text{Percentage Growth} = \left( \frac{\text{Value in 2025} - \text{Value in 2023}}{\text{Value in 2023}} \right) \times 100
\]

### For Net Sales:
- **Net Sales in 2023**: 1189.25 lacs
- **Net Sales in 2025**: 1318.83 lacs

\[
\text{Percentage Growth in Net Sales} = \left( \frac{1318.83 - 1189.25}{1189.25} \right) \times 100 = \left( \frac{129.58}{1189.25} \right) \times 100 \approx 10.91\%
\]

### For Net Profit:
- **Net Profit in 2023**: 85.94 lacs
- **Net Profit in 2025**: 152.16 lacs

\[
\text{Percentage Growth in Net Profit} = \left( \frac{152.16 - 85.94}{85.94} \right) \times 100 = \left( \frac{66.22}{85.94} \right) \times 100 \approx 77.06\%
\]

### Summary:
- **Percentage Growth in Net Sales from 2023 to 2025**: **10.91%**
- **Percentage Growth in Net Profit from 2023 to 2025**: **77.06%**


In [36]:
user_question_9 = "Tell me by analysis all the data how the company is doing"
result_9 = app.invoke({"question": user_question_9})
print(result_9["result"])

Based on the provided Profit and Loss (P&L) data, we can analyze the company's financial performance across several key metrics and ratios. Here’s a detailed analysis:

### 1. **Revenue/Sales**
- **Net Sales Growth**: The company has shown a significant increase in net sales from Rs. 599.96 lacs in 2022 to Rs. 1189.25 lacs in 2023, indicating a growth of approximately 98.2%. The projected sales continue to grow steadily, reaching Rs. 1710.82 lacs by 2029.
- **Trend Analysis**: The growth trend is positive, suggesting effective sales strategies and possibly increased market demand.

### 2. **Cost of Goods Sold (COGS)**
- **COGS Analysis**: The COGS has increased from Rs. 466.95 lacs in 2022 to Rs. 927.80 lacs in 2023. This increase is in line with the growth in sales, but it is essential to monitor the efficiency of production cost management.
- **Efficiency**: The company needs to ensure that the increase in COGS does not outpace revenue growth to maintain healthy profit margins.

### 

In [37]:
user_question_10 = "Tell me that will all the data, can i Provide Loan to the company if loan possiable tell me the loan amount in INR or give me the lone rejection reason"
result_10 = app.invoke({"question": user_question_10})
print(result_10["result"])

To determine whether to provide a loan to the company, we need to analyze the financial metrics and ratios derived from the Profit and Loss (P&L) statement provided. Here are the key considerations:

### Key Financial Metrics Analysis

1. **Revenue Growth**:
   - The company shows a significant increase in net sales from Rs. 599.96 lacs in 2022 to Rs. 1189.25 lacs in 2023, indicating strong revenue growth. This trend continues with projected revenues increasing each year.

2. **Gross Profit Margin**:
   - Gross Profit = Net Sales - Cost of Sales
   - For 2023: Gross Profit = 1189.25 - 927.80 = Rs. 261.45 lacs
   - Gross Profit Margin = (Gross Profit / Net Sales) = (261.45 / 1189.25) * 100 ≈ 22.0%
   - A gross profit margin of 22% indicates that the company is managing its production costs effectively relative to its sales.

3. **Operating Profit Margin**:
   - Operating Profit = Gross Profit - Operating Expenses
   - For 2023: Operating Profit = 261.45 - (Interest + SG&A) = 261.45 - (1

In [38]:
user_question_11 = "What is the over all COGS for Actual and Estimated Years"
result_11 = app.invoke({"question": user_question_11})
print(result_11["result"])

To calculate the overall Cost of Goods Sold (COGS) for the Actual and Estimated years, we will sum the COGS values for each year from the provided Profit and Loss data.

From the data, the COGS is represented in the row labeled "Cost of Sales" (row 23). The values for COGS for the Actual and Estimated years are as follows:

- **Actual Years:**
  - 31.03.2022: 466.95 lacs
  - 31.03.2023: 927.80 lacs

- **Estimated Years:**
  - 2024-03-31: 958.94 lacs
  - 2025-03-31: 967.69 lacs
  - 2026-03-31: 978.09 lacs
  - 2027-03-31: 990.74 lacs
  - 2028-03-31: 1005.44 lacs
  - 2029-03-31: 1021.84 lacs

Now, let's calculate the total COGS for the Actual and Estimated years:

### Total COGS Calculation:

**Actual Years:**
- Total COGS (Actual) = 466.95 + 927.80 = 1394.75 lacs

**Estimated Years:**
- Total COGS (Estimated) = 958.94 + 967.69 + 978.09 + 990.74 + 1005.44 + 1021.84
- Total COGS (Estimated) = 958.94 + 967.69 + 978.09 + 990.74 + 1005.44 + 1021.84 = 5992.74 lacs

### Summary:
- **Total COGS 

In [39]:
user_question_12 = "What is the Gross Sales Growth Rate"
result_12 = app.invoke({"question": user_question_12})
print(result_12["result"])

To calculate the Gross Sales Growth Rate, we can use the formula:

\[
\text{Gross Sales Growth Rate} = \left( \frac{\text{Current Year Gross Sales} - \text{Previous Year Gross Sales}}{\text{Previous Year Gross Sales}} \right) \times 100
\]

From the provided Profit and Loss data, we can extract the Gross Sales figures for the years 2022 and 2023:

- Gross Sales for 31.03.2022: 689.3 lacs
- Gross Sales for 31.03.2023: 1366.23 lacs

Now, we can plug these values into the formula:

\[
\text{Gross Sales Growth Rate} = \left( \frac{1366.23 - 689.3}{689.3} \right) \times 100
\]

Calculating the difference:

\[
1366.23 - 689.3 = 676.93
\]

Now, divide by the previous year's gross sales:

\[
\frac{676.93}{689.3} \approx 0.980
\]

Finally, multiply by 100 to get the percentage:

\[
\text{Gross Sales Growth Rate} \approx 0.980 \times 100 \approx 98.0\%
\]

Thus, the Gross Sales Growth Rate from 31.03.2022 to 31.03.2023 is approximately **98.0%**.


In [40]:
user_question_13 = "Analyse the PL Data and give me the only provide Return on Equity (ROE) calculations if applicable?"
result_13 = app.invoke({"question": user_question_13})
print(result_13["result"])

To calculate Return on Equity (ROE), we need to know the Net Income and the Shareholder's Equity. The formula for ROE is:

\[ \text{ROE} = \frac{\text{Net Income}}{\text{Shareholder's Equity}} \times 100 \]

From the provided Profit and Loss (P&L) data, we can extract the Net Income for the relevant years:

- **Net Income for 31.03.2023**: 85.94 lacs
- **Net Income for 31.03.2024**: 114.38 lacs
- **Net Income for 31.03.2025**: 152.16 lacs
- **Net Income for 31.03.2026**: 187.65 lacs
- **Net Income for 31.03.2027**: 225.45 lacs
- **Net Income for 31.03.2028**: 260.42 lacs
- **Net Income for 31.03.2029**: 291.98 lacs

However, the Shareholder's Equity is not provided in the data. Without the Shareholder's Equity figure, we cannot calculate the ROE.

If you have the Shareholder's Equity values for the corresponding years, please provide them, and I can calculate the ROE for you.
